In [123]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [124]:
# Set executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/trina/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


In [125]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [126]:
# HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text') #look for the <div /> tag and its descendent (select_one returns the first match)

In [127]:
# scraping through slide_elem to find all the div tags with class 'content_title'
slide_elem.find('div', class_='content_title')


<div class="content_title">Screening Soon: 'The Pathfinders' Trains Lens on Mars</div>

In [128]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text() #gets just the text (i.e. in above result, all HTML stuff included)
news_title

"Screening Soon: 'The Pathfinders' Trains Lens on Mars"

In [129]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'With the Mars 2020 mission ramping up, the documentary — the first of four about past JPL missions to the Red Planet to be shown at Caltech — tells a gripping backstory.'

### Featured Images

In [130]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [131]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1] #indexing to stipulate to click second button
full_image_elem.click()

In [132]:
# The above code loads a new page, so we need to parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [133]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [134]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

### Mars Facts

In [135]:
df = pd.read_html('https://galaxyfacts-mars.com')[0] #creates new df from HTML table
df.columns=['description', 'Mars', 'Earth'] #assigning columns
df.set_index('description', inplace=True) #set description column as the index
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [136]:
#convert df back to HTML ready code
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [141]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'
browser.visit(url)


In [142]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
# List all image links
links = browser.find_by_css('a.product-item img')

# Loop through image URLs
for i in range(len(links)):
    hemisphere = {}
    
    # Loop through elements
    browser.find_by_css('a.product-item img')[i].click()
    
    # Get the href for image URLs
    sample_imgs = browser.links.find_by_text('Sample').first
       
    # Get Hemisphere titles
    hem_title = browser.find_by_css('h2.title').text
    
    # Add to hemisphere dictionary
    hemisphere['img_url'] = sample_imgs['href']
    hemisphere['title'] = hem_title  
    
    # Add hemisphere to hemisphere_image_urls
    hemisphere_image_urls.append(hemisphere)
    
    # Navigate back to the beginning to get the next hemisphere image
    browser.back()

In [143]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [144]:
# 5. Quit the browser
browser.quit()